# Fedbiomed Researcher to train a federated PPCA (Probabilistic PCA) model.

## Description of the exercise :

Three datasets `n1.csv` , `n2.csv` and `n3.csv` will be generated randomly using 3-views PPCA from a 4-dimensional latent space, with views dimensions [15,8,10] and 2 groups. Henceforth, we will distribute the 3 dataset to 3 distinct nodes and use Fed-mv-PPCA. In each center we check the evolution of expected LL during training.

## Data Generation

We will generate three datasets using mv-PPCA.
Then save them in a path of your choice on your machine.

In [12]:
import numpy as np
import pandas as pd
from typing import List, Union, Dict

def sample_x_n(N:int, q:int, random_state:int=None):
    """samples from a Guassian dsitribution
    Args:
    
    :N first dimension array
    :q second dimesnion array
    """
    return np.random.RandomState(random_state).randn(N,q)

def generate_data(N_g: List[int],
                  W: np.ndarray,
                  a_g: np.ndarray,
                  mu:float,
                  sigma2:float,
                  x_n,
                  view:int,
                  random_state=None):
    
    """generates Gaussian dataset given several groups of data points, using the following
    gausian generative proccess (for a given view):
    
    Y = WX + mu + epsilon with epsilon ~ N(0, sigma2)
    
    where X is the latent space of size (q, n_features), Y the observation matrix, W the matrix used
    for data reconstructionY
    
    Params:
    :N_g: List[int] number of data to generate per group (list of size number of group)
    :W: reconstruction matrix, of size (n_features, q)
    :a_g: (np.ndarray) array of size (nb_group, n_components), introduces shift when creating different group. 
    :mu: offset of the dataset
    :sigma2: variance used for generating
    :x_n: random variable 
    :view: (int) the given view
    :random_state: ransom seed for reproducibility
    
    Returns:
    :Y (pd.DataFrame): synthetic dataset generated wrt above expression
    of size (n_features, n_samples)
    """
    rnd=np.random.RandomState(random_state)

    N=N_g.sum()
    d, q = W.shape
    sigma=np.sqrt(sigma2)
    G=len(N_g)

    g_ind=np.concatenate((np.zeros(1, dtype=np.int64), np.cumsum(N_g)))
    

    y_n=np.empty((N, d))

    for g in range(G):
        # computing Y = W.transpose(X + shift()) + mu
        y_n[g_ind[g]:g_ind[g+1]]= np.einsum("dq,nq->nd", W, x_n[g_ind[g]:g_ind[g+1]]+a_g[g]) + mu
        
    y_n = pd.DataFrame(data=y_n,
                     columns=[f'var_{view},{i + 1}' for i in range(d)])

    return y_n + sigma*rnd.randn(N,d)



def generate_ppca_nodes_dataset(n_nodes: int,
                           n_features: Union[List[int],int],
                           
                           n_components: int,
                               n_group: int=2,
                           absent_view: Dict[str, int]=None,
                           W_init: List[np.ndarray]=None,
                           mu_init: List[np.ndarray]=None,
                           sigma_init : List[np.ndarray]=None,
                           is_validation: bool=True,
                           n_sample_validation: int=None):
    """
    Generate a synthetic dataset for each node
    
    """
    # generate PPCA parameters if not defined
    ## case where W parameter is not defined
    if W_init is None:
        W_init = []
        for i in range(n_nodes):
            W_gen = np.random.uniform(-10, 10, (n_features[i], n_components))
            W_init.append(W_gen)
    ## case where mu not generated
    if mu_init is None:
        mu_init = []
        for i in range(n_nodes):
            mu_gen = np.random.uniform(-10, 10, n_features[i])
            mu_init.append(mu_gen)
            
    ## case where sigma is not definied (we will set sigma =1 for each clients)
    if sigma_init is None:
        sigma_init = []
        for i in range(n_nodes):
            sigma_init.append(1)
            
    
    shift = np.concatenate((np.zeros((1, n_components)),
                          np.random.uniform(-10, 10, (nb_group - 1, n_components))))

In [13]:
from typing import Dict, Union

def create_multi_view_dataframe(datasets: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    _header_labels = ['views', 'feature_name']
    # 1. create multiindex header
    
    _feature_name_array = np.array([])  # store all feature names
    _view_name_array = []  # store all views (ie modalities) names
    
    _concatenated_datasets = np.array([])  # store dataframe values
    
    for key in datasets.keys():
        #_sub_dataframe_header.append(list(datasets[key].columns.values))
        _feature_name_array = np.concatenate([_feature_name_array,
                                              datasets[key].columns.values])
        if len(_concatenated_datasets) <= 0:
            # first pass 
            _concatenated_datasets = datasets[key].values
        else:
            # other pass
            try:
                _concatenated_datasets = np.concatenate([_concatenated_datasets,
                                                         datasets[key].to_numpy()],
                                                        axis=1)
            except ValueError as val_err:
                # catching case where nb_samples are differents
                raise ValueError('Cannot create multi view dataset: different number of samples for each modality have been detected')
        for _ in datasets[key].columns.values:
            _view_name_array.append(key)

    _header = pd.MultiIndex.from_arrays([_view_name_array, _feature_name_array],
                                        names=_header_labels)
    
    
    
    
    # 2. create multi index dataframe
    
    mulit_view_df = pd.DataFrame(_concatenated_datasets,
                                 columns = _header)
    return mulit_view_df


def save_multi_view_dataframe(dataframe: pd.DataFrame, file_name: str):
    dataframe.to_csv(file_name)
    
def load_multi_view_dataframe(file_name: str) -> pd.DataFrame:
    df = pd.read_csv(file_name, delimiter=',', index_col=0, header=[0,1])
    return df




In [14]:
np.random.seed(100)

D_i = [15, 8, 10]
nb_group = 2
n_centers = 3
testing_samples = 40

n_components_generated = 4

In [15]:



# initializing PPCA variables
sigma2_gen1, sigma2_gen2, sigma2_gen3 = 2, 1, 3
W_gen1 = np.random.uniform(-10, 10, (D_i[0], n_components_generated))
W_gen2 = np.random.uniform(-5, 5, (D_i[1], n_components_generated))
W_gen3 = np.random.uniform(-15, 15, (D_i[2], n_components_generated))
mu_gen1 = np.random.uniform(-10, 10, D_i[0])
mu_gen2 = np.random.uniform(-5, 5, D_i[1])
mu_gen3 = np.random.uniform(-15, 15, D_i[2])

a_g_gen = np.concatenate((np.zeros((1, n_components_generated)),
                          np.random.uniform(-10, 10, (nb_group - 1, n_components_generated))))

W = [W_gen1,W_gen2,W_gen3]
mu = [mu_gen1,mu_gen2,mu_gen3]
sigma = [sigma2_gen1,sigma2_gen2,sigma2_gen3]

# absent_views contains as key the id of a center in which we want to simulate absent views,
# and as argumet the id of the missing view. 
absent_views = {'2': 2}

for i in range(n_centers):
    N_g = np.array([np.random.randint(25,300) for _ in range(nb_group)]) 
    # N_g = np.array([np.random.randint(25,300) for _ in range(nb_group)])?
    #g_ind = np.concatenate((np.zeros(1, dtype=np.int64), np.cumsum(N_g)))
    N = N_g.sum()
    x_n_gen = sample_x_n(N, n_components_generated, random_state=150)  # randomly generate a Gaussian
    # dataset
    #Y = []
    Y = {}
    for d in range(len(D_i)):
        y_t = generate_data(N_g, W[d], a_g_gen, mu[d], sigma[d], x_n_gen, view = d+1, random_state=250)
        if ((str(i+1) in absent_views.keys()) \
            and (type(absent_views[str(i+1)])== int) \
            and (absent_views[str(i+1)]==d+1)):
            absent_views.update({str(i+1): y_t})
            y_abs=pd.DataFrame(np.nan, index = np.arange(N_g.sum()), \
                               columns = [f'var_{d+1},{i + 1}' for i in range(D_i[d])])
            #Y.append(y_abs)
            Y['view_' + str(d+1)] = y_abs
        else:
            # Y.append(y_t)
            Y['view_' + str(d+1)] = y_t

    gr = []
    for g in range(nb_group):
       gr += [int(g) for _ in range(N_g[g])]
    gr = pd.Series(gr)
    Y['Label'] = pd.DataFrame(gr, columns=['Labels'])

###
# the output will be a list of n_centers dataset containing: dataframe for each centers,of 
# different dimensions
# 
    #t_i = pd.concat(Y, axis=1)
    
    #t_i.columns.values[-1] = 'Label'
    t_i = create_multi_view_dataframe(Y)
    t_i.to_csv('== Local path to node' + str(i+1) + '.csv',sep=',')
    #np.savetxt('== Local path to node' + str(i+1) + '.csv',t_i,delimiter=',')
               
# building the test dataset
N_g_test = np.array([testing_samples//2,testing_samples//2])
g_ind_test = np.concatenate((np.zeros(1, dtype=np.int64), np.cumsum(N_g_test)))
N_test = N_g_test.sum()
x_n_gen = sample_x_n(N_test, n_components_generated, random_state=150)
Y_test = {}
for d in range(len(D_i)):
    y_t = generate_data(N_g_test, W[d], a_g_gen, mu[d], sigma[d], x_n_gen, view = d+1, random_state=250)
    Y_test['view_' + str(d+1)] = y_t

gr_test = [0 for _ in range(N_g_test[0])]+[1 for _ in range(N_g_test[1])]
gr_test = pd.Series(gr_test)
Y_test['Label'] = pd.DataFrame(gr_test, columns=['Labels'])

t_test = pd.concat(Y_test, axis=1)

In [16]:
t_test

view_1                                                          \
      var_1,1    var_1,2     var_1,3    var_1,4    var_1,5    var_1,6   
0    9.310768  -6.461914   -5.620688  -4.694115  -0.722527  15.265315   
1   -5.682923 -29.503222   -1.634534 -29.912643  -4.597543  33.058772   
2    9.481735 -16.868594  -17.183134  -0.456770  -5.737701   3.158270   
3  -12.908970 -20.203461   24.409138 -43.366352  10.613558  45.681332   
4    4.104555 -30.477031  -16.065672 -23.631253  31.544356   1.278780   
5    1.387530 -18.059214   -1.742302 -18.667190  20.036034   1.161863   
6   15.235876  15.680027    2.961760   7.765629  17.181556   2.913996   
7    9.501694  21.040584   13.677487   9.533322 -16.485215  19.801435   
8    2.749107 -15.182634   -4.110620 -13.122069   2.287465   9.128295   
9   15.009937   1.609956  -11.618127  -0.746035   5.656150  14.440896   
10   6.970560 -22.600734  -19.800251 -10.061976  -9.075024  18.275109   
11   7.124169  -9.965620  -17.748917   7.580806  -8.537070  -3.552130   
12   4.921071  -9.024447   -0.265654 -10.841773   9.625035   9.599857   
13  10.617188   2.525194    3.378305  -4.057518   8.581564  15.528880   
14   7.849417 -27.354317  -20.538884 -16.042236   9.834178   1.629209   
15  -8.194118 -31.914798   -3.136441 -29.821013   4.126672  19.180894   
16   8.774830  -2.163111  -10.945446   8.214751 -16.767999   5.286502   
17  -6.578779   2.568367   21.941211 -11.372428   8.997729  20.168920   
18  13.179485   3.012626   -4.591592   4.187134   7.170097   4.602894   
19  -8.051734 -20.460908   14.189010 -29.141483  -2.475129  37.813774   
20  43.625186 -49.579204  -95.891356  17.745783   5.611837 -43.398736   
21  33.040333 -31.957876  -72.510840  22.886127  20.264293 -42.155369   
22  43.964907 -31.669703  -85.623773  23.007175  27.000120 -46.400761   
23  44.555674 -45.621724  -90.427666   9.535623  41.330724 -47.816565   
24  61.408917  -9.845936  -90.882469  38.414527  32.679154 -58.242647   
25  44.663760 -24.297859  -74.311483  19.469356  35.683718 -41.785194   
26  50.294422 -45.489440 -102.492505  27.218624  12.757323 -48.818045   
27  65.138369 -35.215518 -123.469640  48.336463  13.522215 -69.130845   
28  64.212400 -21.595326 -106.909544  42.699906  17.028344 -59.148081   
29  52.509659 -30.178824 -100.737719  39.899623 -11.649944 -42.374701   
30  43.109119 -32.748121  -82.599402  20.586592  23.601058 -42.770690   
31  72.847997   1.658675  -98.788777  55.953445  34.642665 -67.687034   
32  36.991400 -33.076281  -74.693962  21.018225   9.131397 -36.254011   
33  57.770181  10.233937  -71.917393  53.036670  22.155487 -50.930019   
34  55.256587 -32.328786 -109.526731  44.293917  23.727408 -75.260800   
35  55.042622 -13.579502  -92.243243  45.622506  14.530123 -51.466310   
36  50.773174 -22.023032  -86.450522  37.980370   8.992954 -47.198140   
37  45.313480 -33.747292  -83.211332  20.463989   6.775574 -28.443796   
38  41.025307 -50.881948  -85.961000   4.710952  37.852265 -44.676329   
39  63.072239 -19.971643 -105.591132  52.794701   5.751033 -67.011180   

                                                ...     view_3             \
      var_1,7    var_1,8    var_1,9   var_1,10  ...    var_3,2    var_3,3   
0    1.472637   7.819969  -4.907187   3.072993  ...   3.615410  -0.257516   
1    5.594790  -8.081894 -25.082754 -12.439787  ...   2.569041  23.109628   
2   27.514549   6.222885   2.348626  -9.865517  ...  12.566323  -3.256145   
3  -14.135197  -4.561476 -44.835105   3.176304  ... -18.565325  25.181532   
4   13.459440  31.989547 -34.999132  30.678031  ... -25.503408  13.981502   
5   20.005699  20.287200 -25.576522  14.804430  ... -19.027170   6.584444   
6    4.662045  26.245601 -11.930605  22.583935  ... -11.309408 -18.603111   
7   12.480652  -8.527035  10.451323 -22.170635  ...  25.166966 -19.779914   
8   18.998826   3.619119 -12.691160   0.910401  ...  -0.748995   0.610854   
9   -9.700489  15.264701 -10.967147  17.428055  ...  -5.428858  -4.499190   
10   1.050565   

In [17]:

load_multi_view_dataframe('== Local path to node2.csv')['view_2']

feature_name,"var_2,1","var_2,2","var_2,3","var_2,4","var_2,5","var_2,6","var_2,7","var_2,8"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Start the network and setting the client up
Before running this notebook:
1. You should start the network from fedbiomed-network, as detailed in :
https://gitlab.inria.fr/fedbiomed/fedbiomed

2. You need to configure at least 2 nodes: <br/>
* **Node 1 :** `./scripts/fedbiomed_run node add`
  * Select option 1 to add a csv file to the client
  * Choose the name, tags and description of the dataset (you can write 'sk' always and it will be good)
  * Pick the .csv file you stored the couple X[0],y[0].
  * Check that your data has been added in node 1 by executing `./scripts/fedbiomed_run node list`
  * Run the node using `./scripts/fedbiomed_run node start`. <br/>

* **Node 2 :** Open a second terminal and run ./scripts/fedbiomed_run node add config n2.ini
  * Select option 1 to add a csv file to the client
  * Choose the name, tags and description of the dataset (you can write 'sk' always and it will be good)
  * Pick the .csv file you stored the couple X[1],y[1].
  * Check that your data has been added in node 2 by executing `./scripts/fedbiomed_run node config n2.ini list`
  * Run the node using `./scripts/fedbiomed_run node config n2.ini start`.
  


 Wait until you get `Connected with result code 0`. it means node is online.


In [18]:
%load_ext autoreload
%autoreload 2

In [19]:
import numpy as np
from fedbiomed.researcher.environ import TMP_DIR
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=TMP_DIR+'/')
model_file = tmp_dir_model.name + '/fed_mv_ppca.py'

Hereafter the template of the class you should provide to Fedbiomed :
       
**training_data** : you must return here a tuple (X,X_k,ViewsX,y) or (X,X_k,ViewsX). Note that all centers should provide a dataset with the same view-specific columns. If in a specific center a view has not been observed, then the corresponding columns will be filled of nan. The training_data method take care of identifying view-specific sub-datasets and collecting information concerning non-available observations. Data can also been normalized here.

In [20]:
%%writefile "$model_file"

from fedbiomed.common.ppca import PpcaPlan
import numpy as np
import pandas as pd


class Fed_MV_PPCA(PpcaPlan):
    def __init__(self, kwargs):
        super(Fed_MV_PPCA, self).__init__(kwargs)
        deps = ['import numpy as np', 
               'import pandas as pd']
        self.add_dependency(deps)
        self.multi_view = True
    
    def training_data(self):
        """
            Perform in this method all data reading and data transformations you need.
            At the end you should provide a tuple (X_obs,Xk,ViewsX,y), where: 
            X_obs is the training dataset, 
            Xk is a list containing the k-specific dataframe if it exists or 'NaN' otherwise,
            ViewsX is the indicator function for observed views (ViewsX[k]=1 if view k is observed, 0 otherwise)
            y the corresponding labels (optional)
            The dataset is normalized using min max scaler if model_args['norm'] is true
            Note: since labels are not needed for the optimization, 
            training_data can also simply return (X_obs,Xk,ViewsX)
            :raise NotImplementedError if researcher do not implement this method.
        """
        #dataset = pd.read_csv(self.dataset_path,delimiter=',', index_col=0)
        dataset = self.load_multi_view_dataframe(self.dataset_path)
        X = dataset.iloc[:,:-1]
        y = dataset[dataset.columns[-1]]
        
        # Xk is a list contistripaning the view-specific local datasets
        Xk = []
        ViewsX = []
        ind = 0
        for k in range(self.K):
            if X.iloc[:, ind].isnull().values.any():
                Xk.append(np.nan)
                ViewsX.append(0)
            else:
                # if norm = true, data are normalized with min max scaler
                X_k = X.iloc[:, ind:ind + self.dim_views[k]]
                if self.is_norm:
                    X_k = self.normalize_data(X_k) 
                    
                Xk.append(X_k)
                ViewsX.append(1)
            ind += self.dim_views[k]
        
        # The entire dataset is re-built without empty columns
        Xk_obs = [item for item in Xk if item is not np.nan]
        X_obs = pd.concat(Xk_obs, axis=1)
        
        return (X_obs,Xk,ViewsX,y)
    
    def load_multi_view_dataframe(self, file_name: str) -> pd.DataFrame:
        df = pd.read_csv(file_name, delimiter=',', index_col=0, header=[0,1])
        return df
    

Writing /home/ybouilla/fedbiomed/var/tmp/tmp0sgvedb6/fed_mv_ppca.py


**model_args** is a dictionary containing the mv-ppca model arguments: the total number of views across all datasets (tot_views), the dimension of each view (dim_views), the latent space size (n_components), and a boolean (norm) for data preprocessing. Additionaly, the researcher can provide priors for one ore more global parameters.

**training_args** contains here the number of local iterations for EM/MAP. 

In [21]:
tot_views = 3
dim_views = [15, 8, 10]
n_components = 4
norm = True

model_args = {'tot_views': tot_views, 'dim_views': dim_views, 'n_components': n_components, 'is_norm': norm}

training_args = {'n_iterations': 15}

tags =  ['ppca_data']
rounds = 5

In [22]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.ppca_aggregator import MLaggregator



# select nodes pr into task <Task pending name='Task-27' coro=<HTTP1ServerConnection._server_request_loop() running at /user/ybouilla/home/.conda/envs/fedbiomed-researcher/lib/python3.9/site-packages/tornado/http1connection.py:823> wait_for=<Future finished result=b'GET /kernel...6bd7"\r\n\r\n'> cb=[IOLoop.add_future.<locals>.<lambda>() at /user/ybouilla/home/.conda/envs/fedbiomed-researcher/lib/python3.9/site-packages/tornado/ioloop.py:688]> while another task <Task pending name='Task-2' coro=<KernelManager._async_start_kernel() running at /user/ybouilla/home/.conda/envs/fedbiomed-researcher/lib/python3.9/site-packages/jupyter_carticiping to this experiment
exp = Experiment(tags=tags,
                 #clients=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='Fed_MV_PPCA',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=MLaggregator(),
                 client_selection_strategy=None)

2021-11-03 18:42:09,057 fedbiomed INFO - Messaging researcher_05c21db4-de54-45f7-a0a7-e84d3f770fd6 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7fbec9dde9a0>
2021-11-03 18:42:09,074 fedbiomed INFO - Searching dataset with data tags: ['ppca_data'] for all nodes
2021-11-03 18:42:09,075 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - DEBUG Message received: {'researcher_id': 'researcher_05c21db4-de54-45f7-a0a7-e84d3f770fd6', 'tags': ['ppca_data'], 'command': 'search'}
2021-11-03 18:42:09,076 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - DEBUG Message received: {'researcher_id': 'researcher_05c21db4-de54-45f7-a0a7-e84d3f770fd6', 'tags': ['ppca_data'], 'command': 'search'}
2021-11-03 18:42:19,092 fedbiomed INFO - Checking data quality of federated datasets...
2021-11-03 18:42:19,197 fedbiomed INFO - Messaging NodeTrainingFeedbackClient successfully connected to the message broker,

In [ ]:
from fedbiomed.researcher.requests import Requests

req = Requests()
datasets = req.list(verbose=True)

In [23]:
# start federated training
exp.run()

2021-11-03 18:42:20,624 fedbiomed INFO - Sampled clients in round 0 ['client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34', 'client_fdf81fad-e120-473b-8d04-5cd3a44a7b78']
2021-11-03 18:42:20,624 fedbiomed INFO - Send message to client client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - {'researcher_id': 'researcher_05c21db4-de54-45f7-a0a7-e84d3f770fd6', 'job_id': 'd3cb67ea-c8b4-4305-b998-d2d86d569d51', 'training_args': {'n_iterations': 15}, 'model_args': {'tot_views': 3, 'dim_views': [15, 8, 10], 'n_components': 4, 'is_norm': True}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2021/11/03/my_model_7297d9c1-be4b-47ae-92c5-f2252ced2a8d.py', 'params_url': 'http://localhost:8844/media/uploads/2021/11/03/my_model_398e7b8a-5f16-4d13-a05e-9d577847f771.pt', 'model_class': 'Fed_MV_PPCA', 'training_data': {'client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34': ['dataset_56df3e9a-8d17-44f9-a09d-a2b6e864f255']}}
2021-11-03 18:42:20,625 fedbiomed DEBUG - researcher_05c21db4-de54-45f7-a0a7-e84d3f7

2021-11-03 18:42:22,163 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - INFO Iteration: 9/15	Expected LL: 9940.736703
2021-11-03 18:42:22,183 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - INFO Iteration: 9/15	Expected LL: 9305.150059
2021-11-03 18:42:22,289 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - INFO Iteration: 10/15	Expected LL: 9585.119553
2021-11-03 18:42:22,350 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - INFO Iteration: 10/15	Expected LL: 10427.452897
2021-11-03 18:42:22,410 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - INFO Iteration: 11/15	Expected LL: 9827.000031
2021-11-03 18:42:22,528 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - INFO Iteration: 12/15	Expected LL: 10038.373492
2021-11-03 18:42:22,554 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - INFO Iteration: 11/15	Expected LL: 1088

2021-11-03 18:42:30,735 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - DEBUG Dataset_path/home/ybouilla/fedbiomed/notebooks/== Local path to node2.csv
2021-11-03 18:42:30,737 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - DEBUG Dataset_path/home/ybouilla/fedbiomed/notebooks/== Local path to node1.csv
2021-11-03 18:42:30,739 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - DEBUG is Dataset  multi view ? True
2021-11-03 18:42:30,740 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - DEBUG is Dataset  multi view ? True
2021-11-03 18:42:30,952 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - INFO Iteration: 1/15	Expected LL: 10090.917408
2021-11-03 18:42:30,988 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - INFO Iteration: 1/15	Expected LL: 11674.028762
2021-11-03 18:42:31,165 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b

2021-11-03 18:42:40,776 fedbiomed DEBUG - researcher_05c21db4-de54-45f7-a0a7-e84d3f770fd6
2021-11-03 18:42:40,778 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - DEBUG [TASKS QUEUE] Item:{'researcher_id': 'researcher_05c21db4-de54-45f7-a0a7-e84d3f770fd6', 'job_id': 'd3cb67ea-c8b4-4305-b998-d2d86d569d51', 'params_url': 'http://localhost:8844/media/uploads/2021/11/03/researcher_params_3a0a1536-283d-4cab-8f43-f6d58f50f4e0.pt', 'training_args': {'n_iterations': 15}, 'training_data': {'client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34': ['dataset_56df3e9a-8d17-44f9-a09d-a2b6e864f255']}, 'model_args': {'tot_views': 3, 'dim_views': [15, 8, 10], 'n_components': 4, 'is_norm': True}, 'model_url': 'http://localhost:8844/media/uploads/2021/11/03/my_model_7297d9c1-be4b-47ae-92c5-f2252ced2a8d.py', 'model_class': 'Fed_MV_PPCA', 'command': 'train'}
2021-11-03 18:42:40,783 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - DEBUG Message received: {'researcher_id

2021-11-03 18:42:50,819 fedbiomed INFO - Downloading model params after training on client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - from http://localhost:8844/media/uploads/2021/11/03/node_params_d38cf8c1-9300-47b7-bbf0-ec94c5fafe28.pt
2021-11-03 18:42:50,840 fedbiomed INFO - Clients that successfully reply in round 2 ['client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34', 'client_fdf81fad-e120-473b-8d04-5cd3a44a7b78']
2021-11-03 18:42:50,868 fedbiomed INFO - Sampled clients in round 3 ['client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34', 'client_fdf81fad-e120-473b-8d04-5cd3a44a7b78']
2021-11-03 18:42:50,868 fedbiomed INFO - Send message to client client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - {'researcher_id': 'researcher_05c21db4-de54-45f7-a0a7-e84d3f770fd6', 'job_id': 'd3cb67ea-c8b4-4305-b998-d2d86d569d51', 'training_args': {'n_iterations': 15}, 'model_args': {'tot_views': 3, 'dim_views': [15, 8, 10], 'n_components': 4, 'is_norm': True}, 'command': 'train', 'model_url': 'http://localhost:8844/media/

2021-11-03 18:42:52,654 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - INFO Iteration: 7/15	Expected LL: 22259.549954
2021-11-03 18:42:52,787 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - INFO Iteration: 8/15	Expected LL: 17103.178509
2021-11-03 18:42:52,847 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - INFO Iteration: 8/15	Expected LL: 22070.356933
2021-11-03 18:42:52,997 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - INFO Iteration: 9/15	Expected LL: 16907.574892
2021-11-03 18:42:53,033 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - INFO Iteration: 9/15	Expected LL: 21880.670565
2021-11-03 18:42:53,188 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - INFO Iteration: 10/15	Expected LL: 21691.398027
2021-11-03 18:42:53,235 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - INFO Iteration: 10/15	Expected LL: 166

2021-11-03 18:43:00,976 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - INFO {'monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x7f0a2b6b1a90>, 'n_iterations': 15}
2021-11-03 18:43:00,977 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - DEBUG Dataset_path/home/ybouilla/fedbiomed/notebooks/== Local path to node2.csv
2021-11-03 18:43:00,982 fedbiomed INFO - log from: client_5c14f813-81c9-4ee8-baa0-6d27e27e8c34 - DEBUG is Dataset  multi view ? True
2021-11-03 18:43:00,988 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - INFO {'monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x7f0385afe610>, 'n_iterations': 15}
2021-11-03 18:43:00,989 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - DEBUG Dataset_path/home/ybouilla/fedbiomed/notebooks/== Local path to node1.csv
2021-11-03 18:43:00,991 fedbiomed INFO - log from: client_fdf81fad-e120-473b-8d04-5cd3a44a7b78 - 

In [24]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())


List the training rounds :  dict_keys([0, 1, 2, 3, 4])

Access the federated params for the last training round :
	- params_path:  /home/ybouilla/fedbiomed/var/tmp/researcher_params_22ea52e1-c030-4ac0-8d0b-eda90eac4e17.pt
	- parameter data:  dict_keys(['tilde_muk', 'tilde_Wk', 'tilde_Sigma2k', 'Alpha', 'Beta', 'sigma_til_muk', 'sigma_til_Wk', 'sigma_til_sigma2k'])


## Test

Herafter we test the performance of the aggregated parameters on a test dataset. In particular, for each round we use the global parameters to evaluate the mean absolute error and the separation in the latent space using LDA. Note that we have already defined the test dataset at the beginning of this notebook.

In [25]:

import numpy as np
import pandas as pd
from numpy.linalg import solve

from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error




def eval_MB(Wk, q, D_i, K, Sigma2, ViewsX):
    """
    Computes matrices M:=inv(I_q+sum_k Wk.TWk/sigma2k) and B:= [W1.T/sigma2K,...,W1.T/sigma2K].
    :param Wk: list of matrices (d_k x q)
    :param Sigma2: list of float > 0
    :return np.arrays
    """
    index = ViewsX.index(1)

    M1 = Wk[index].reshape(D_i[index], q).T.dot(Wk[index].reshape(D_i[index],q)) / Sigma2[index]
    B = Wk[index].reshape(D_i[index], q).T / Sigma2[index]
    for k in range(index + 1, K):
        if ViewsX[k] == 1:
            M1 += Wk[k].reshape(D_i[k], q).T.dot(Wk[k].reshape(D_i[k],q)) / Sigma2[k]
            B = np.concatenate((B, (Wk[k].reshape(D_i[k], q)).T / Sigma2[k]), axis=1)

    M = solve(np.eye(q) + M1,np.eye(q))

    return M, B

def concat_params(park, K, ViewsX):
    """
    This function concatenates parameters from a list
    :param park: list of vectors/matrices to concatenate
    :return np.array
    """
    index = ViewsX.index(1)

    par = park[index]
    for k in range(index + 1, K):
        if ViewsX[k] == 1:
            par = np.concatenate((par, park[k]), axis=0)

    return par
[]
def simu_latent(q,dataset,ViewsX,global_params):
    """
    This function allows sampling of x_n (latent variables) from the posterior distribution 
    (with global parameters).
    :return pandas dataframe
    """
    d = dataset.shape[1]
    N = dataset.shape[0]
    K = len(ViewsX)

    mu = concat_params(global_params['tilde_muk'], K, ViewsX)
    M, B = eval_MB(global_params['tilde_Wk'], q, D_i, K, global_params['tilde_Sigma2k'],ViewsX)

    Xn = [(M.dot(B).dot(dataset.iloc[n].values.reshape(d, 1) - mu)).reshape(1, q) for n in range(N)]

    df = pd.DataFrame(np.vstack(Xn), index=dataset.index)

    return df

def MAE(dataset,ViewsX,q,global_params):
    """
    This function evaluates the MAE using global parameters
    :return float
    """    
    d = dataset.shape[1]
    N = dataset.shape[0]
    K = len(ViewsX)
    
    mu = concat_params(global_params['tilde_muk'], K, ViewsX)
    W = concat_params(global_params['tilde_Wk'], K, ViewsX)
    M, B = eval_MB(global_params['tilde_Wk'], q, D_i, K, global_params['tilde_Sigma2k'],ViewsX)

    T_true = dataset.values.tolist()

    T_pred = []
    for n in range(N):
        Xng = (M.dot(B).dot(dataset.iloc[n].values.reshape(d, 1) - mu)).reshape(q, 1)
        T_pred.append((W.dot(Xng) + mu).reshape(d))

    MAE = mean_absolute_error(T_true, T_pred)



In [ ]:
df = load_multi_view_dataframe('== Local path to node2.csv')

X = df.iloc[:,:-1]
y = df[df.columns[-1]]

# Xk is a list contistripaning the view-specific local datasets
Xk = []
ViewsX = []
ind = 0
for k in range(3):
    if X.iloc[:, ind].isnull().values.any():
        Xk.append(np.nan)
        ViewsX.append(0)
    else:
        # if norm = true, data are normalized with min max scaler
        X_k = X.iloc[:, ind:ind + dim_views[k]]
        if True is True:
            X_k = ppca.normalize_data(X_k) 

        Xk.append(X_k)
        ViewsX.append(1)
    ind += dim_views[k]

# The entire dataset is re-built without empty columns
Xk_obs = [item for item in Xk if item is not np.nan]
X_obs = pd.concat(Xk_obs, axis=1)

In [ ]:
Xk_obs[1]

In [ ]:
ppca.normalize_data(t_test)

In [26]:
from fedbiomed.common.ppca import PpcaPlan

ppca = PpcaPlan(model_args)
ppca.is_multi_view = True

In [27]:
# Test datasetnp.random.seed(100)

D_i = [15, 8, 10]
nb_group = 2
n_centers = 3
testing_samples = 40

n_components_generated = 4


def test_data(ppca, dataset,norm,K,dim_views):
        """
            Equivalent to training_data, for the test dataset
        """
        X = dataset.iloc[:,:-1]
        y = dataset[dataset.columns[-1]]
        
        # Xk is a list contianing the view-specific local datasets
        Xk = []
        ViewsX = []
        ind = 0
        for k in range(K):
            if X.iloc[:, ind].isnull().values.any():
                Xk.append(np.nan)
                ViewsX.append(0)
            else:
                # if norm = true, data are normalized with min max scaler
                X_k = X.iloc[:, ind:ind + dim_views[k]]
                if norm:
                    X_k = ppca.normalize_data(X_k)
                Xk.append(X_k)
                ViewsX.append(1)
            ind += dim_views[k]
        
        # The entire dataset is re-built without empty columns
        Xk_obs = [item for item in Xk if item is not np.nan]
        X_obs = pd.concat(Xk_obs, axis=1)
        
        return (X_obs,Xk,ViewsX,y)

In [28]:
ppca.is_multi_view

True

In [29]:
from sklearn import preprocessing

for r in range(1,rounds):
    global_params = exp.aggregated_params[r]['params']
    ######## Train data
    MAE_train = []
    Latent_Train = pd.DataFrame()
    Label_Train = pd.Series(dtype='int64')
    for c in range(n_centers):
        dataset_c = pd.read_csv('== Local path to node' + str(c+1) + '.csv', delimiter=',', index_col=0)
        X_obs_c,Xk_c,ViewsX_c,y_c = test_data(ppca, dataset_c,norm,tot_views,dim_views)
        # Dataframe of latent space for LDA
        Latent_Train = Latent_Train.append(simu_latent(n_components,X_obs_c,ViewsX_c,global_params))
        Label_Train = Label_Train.append(y_c)
        # MAE Train
        MAE_train.append(MAE(X_obs_c,ViewsX_c,n_components,global_params))

    

    ######## Test data
    X_obs_test,Xk_test,ViewsX_test,y_test = test_data(t_test,norm,tot_views,dim_views)

    Latent_Test = pd.DataFrame()
    Label_Test = pd.Series(dtype='int64')

    # Dataframe of latent space for LDA
    Latent_Test = Latent_Test.append(simu_latent(n_components,X_obs_test,ViewsX_test,global_params))
    Label_Test = Label_Test.append(y_test)
    # MAE Test
    MAE_test = MAE(X_obs_test,ViewsX_test,n_components,global_params)


    print('Round {}:'.format(r))
    print('MAE train (mean,std) = ({:.4f},{:.4f}) \
    \t MAE test = {:.4f} \
    '.format(np.mean(np.array(MAE_train)), \
                                                 np.std(np.array(MAE_train)), MAE_test))

ValueError: could not convert string to float: 'var_1,1'

In [ ]:
conf_LDA_Test